<a href="https://colab.research.google.com/github/LSDtopotools/lsdtt_notebooks/blob/master/lsdtopotools/lsdtopotools_with_condacolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing LSDTopoTools using condacolab

Last updated by Simon M. Mudd on 12/12/2022

This is a starter notebook for installing LSDTopoTools using condacolab. It is a bit faster than other conda installations. You should allocate around 3 minutes of waiting for these cells to execute and for everything to work. 
These times have gone down significantly in the last year so hopefully in the future you might be able to get all of `lsdtopotools` working in under 2 minutes. 

If you use this software for your research, please cite https://doi.org/10.5281/zenodo.3245040

First install condacolab. You need to execute this before you do anything else. This step takes about 40 seconds. 

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

Now use mamba to install `lsdtopotools`. This will install the command line tools. This step takes a bit over a minute. 

In [ ]:
!mamba install lsdtopotools &> /dev/null

We can test out if it is working:

In [ ]:
!lsdtt-basic-metrics -h

If you use the `-h` flag you get a list of all the possible parameters in a csv file. Here are the first few lines:

In [ ]:
import pandas as pd
help_df = pd.read_csv("lsdtt-basic-metrics-README.csv")
help_df.head(10)

Now we install stuff we need for using geospatial python. This takes a bit under 3 minutes.  

In [ ]:
!mamba install gdal shapely rasterio geopandas fiona pyproj cartopy utm &> /dev/null

Unfortunately there is often a problem with installing pyproj, so we nered to test if that has installed correctly and implement a fix if it has not. To do this we find the file `proj.db` and then insert this into the path.

In [ ]:
!find .. -name "proj.db"

Based on the results of that cell, we then add the file location to the path. The below cell is the location of the file that I found on the 12th of December 2022. This is likeley to change in the future. 

In [ ]:
import os; os.environ['PROJ_LIB'] = '../usr/local/share/proj'

Finally install lsdviztools. Occasionally you need to run this line twice. I have no idea why that is. 

In [ ]:
!pip install lsdviztools

## Check to see if everything is working

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw

Before you grab data using `lsdviztools` you need to upload your API key for `opentopography.org`. You can either copy a text file into the directory or make a text file in colab and then paste the key into that file. In the below cell, make sure the string for `your_OT_api_key_file` reflects the filename of the file you just added to colab. 

In [ ]:
# YOU NEED TO PUT YOUR API KEY IN A FILE
your_OT_api_key_file = "my_OT_api_key.txt"

with open(your_OT_api_key_file, 'r') as file:
    print("I am reading you OT API key from the file "+your_OT_api_key_file)
    api_key = file.read().rstrip()
    print("Your api key starts with: "+api_key[0:4])

This actually downloads some topographic data. 

In [ ]:
Dataset_prefix = "SB"
SB_DEM = bmt.ot_scraper(source = "SRTMGL1",longitude_W = -120.464655, longitude_E = -120.254214, 
                        latitude_S = 34.440538, latitude_N = 34.610770,
                        prefix = Dataset_prefix, api_key_file = your_OT_api_key_file)
SB_DEM.print_parameters()
SB_DEM.download_pythonic()
DataDirectory = "./"
Fname = Dataset_prefix+"_SRTMGL1.tif"
gio.convert4lsdtt(DataDirectory,Fname)

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
r_prefix = Dataset_prefix+"_SRTMGL1_UTM"
w_prefix = Dataset_prefix+"_SRTMGL1_UTM"
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = r_prefix,
                                 write_prefix= w_prefix,
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()
lsdtt_drive.run_lsdtt_command_line_tool()


In [ ]:
%matplotlib inline
Base_file = Dataset_prefix+"_SRTMGL1_UTM"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")